# Chapter 11 - Capstone: Forecasting the number of antidiabetic drug prescriptions in Australia

## 11.1 Importing the required libraries and loading the data

## 11.2 Visualizing the series and its components

## 11.3 Modeling the data

## 11.4 Forecasting and evaluating the model's performance

## Next steps